In [1]:
##########----- Reused code -----##########
# Last update: Sept 13, 2020
# By: Huitian Diao
###########################################

library(stringr)

code_dir <- getwd()
tool_dir <- unlist(strsplit(code_dir,"/"))
tool_dir <- head(tool_dir, length(tool_dir)-2)
tool_dir <- paste(c(tool_dir, "tools"), collapse="/")
source(file.path(tool_dir, "function_GSEA.R"))
base_dir <- gsub("/codes_local", "", code_dir)
base_dir

Loading required package: AnnotationDbi

Loading required package: stats4

Loading required package: BiocGenerics

Loading required package: parallel


Attaching package: ‘BiocGenerics’


The following objects are masked from ‘package:parallel’:

    clusterApply, clusterApplyLB, clusterCall, clusterEvalQ,
    clusterExport, clusterMap, parApply, parCapply, parLapply,
    parLapplyLB, parRapply, parSapply, parSapplyLB


The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


The following objects are masked from ‘package:base’:

    anyDuplicated, append, as.data.frame, basename, cbind, colnames,
    dirname, do.call, duplicated, eval, evalq, Filter, Find, get, grep,
    grepl, intersect, is.unsorted, lapply, Map, mapply, match, mget,
    order, paste, pmax, pmax.int, pmin, pmin.int, Position, rank,
    rbind, Reduce, rownames, sapply, setdiff, sort, table, tapply,
    union, unique, unsplit, which, which.max, which.min


Loading required package: Biobase

[1] "/media/pipkin/ROCKET-PRO/CD8_DEV_SC/0_Acute-Chronic"

## 1. GSEA analysis for each set of comparison

In [2]:
##########----- Parameters-----##########
analysis_name <- c("1", "resampled_ARM-NAV")
genome <- "mm"
#########################################
analysis_subdir <- paste(analysis_name[1], "Scanpy_out", analysis_name[2], sep="_")
sum.dir <- file.path(base_dir, '1_Scanpy', analysis_subdir, "4_DE_GSEA")
dir.create(sum.dir, showWarnings = FALSE, recursive=TRUE)
setwd(sum.dir)

###----- Find all de files
de.out.dir <- file.path(base_dir, '1_Scanpy', analysis_subdir, "2_DE")
de.files <- list.files(de.out.dir, pattern="*differential.csv", full.names=TRUE,recursive=TRUE)

use.cols <- c("gene_names", "t-test_overestim_var_logfc", "t-test_overestim_var_padj")

###----- Select gene signature
gs.dir <- file.path('/media/pipkin/ROCKET-PRO/T_cell_signature_Reference/X_GeneSignatures_mm')
gs.pattern <- paste("_", genome, "_sigs.csv", sep="")
gs.files <- list.files(gs.dir, pattern=gs.pattern, full.names=TRUE)
gs.files <- c('/media/pipkin/ROCKET-PRO/CD8_DEV_SC/9_Figures/2_IL2RAKO_Nascent/IL2_dependent_mm_sigs.csv') #, gs.files
gs.files

[1] "/media/pipkin/ROCKET-PRO/CD8_DEV_SC/9_Figures/2_IL2RAKO_Nascent/IL2_dependent_mm_sigs.csv"

In [3]:
completed.df <- data.frame()
for (gs.file.i in c("file", gs.files)) {
    gs.file.base <- gsub(gs.pattern, "",basename(gs.file.i))
    completed.df[gs.file.base] <- character(0)
}
completed.name <- file.path(sum.dir, "completed.csv")

for (file.i in de.files) {
    file.i.qc.row <- c(file.i)
    
    ###----- Create output names & output directory
    file.i.outpath <- gsub(de.out.dir, "", file.i)
    file.i.outpath <- gsub(".csv", "", file.i.outpath)
    file.i.outpath <- gsub("^/", "",file.i.outpath)

    file.i.simp.name <- tail(unlist(strsplit( file.i.outpath, "/")), 1)
    file.i.simp.name <- gsub("_differential", "", file.i.simp.name)
    file.i.simp.vec <- unlist(strsplit(file.i.simp.name, "_vs_"))

    # Create output directory
    file.i.dir <- file.path(sum.dir, paste(file.i.outpath, "_sep", sep=""))
    dir.create(file.i.dir, recursive=TRUE, showWarnings=FALSE)
    
    for (gs.file.i in gs.files) {
        gs.file.base <- gsub(gs.pattern, "",basename(gs.file.i))
        file.i.gs.dir <- file.path(file.i.dir, gs.file.base)
        dir.create(file.i.gs.dir, recursive=TRUE, showWarnings=FALSE)
        setwd(file.i.gs.dir)
        
        # Run GSEA analysis
        useGroup <- "t-test_overestim_var_logfc"
        outName <- file.i.simp.name
        
        success <- "yes"
        gsea <- tryCatch({
            GSEA_analysis(file.i, useGroup, outName, gs.file.i, 1)
        }, error=function(cond) {
            message(paste(file.i, gs.file.i,sep='; '))
            return("no")
        })
        
        if (!is.null(gsea)){
            success <- ""
        }
        
        file.i.qc.row <- c(file.i.qc.row, success)
    }
    
    completed.df[nrow(completed.df) +1, ] <- file.i.qc.row
    write.csv(completed.df, completed.name, row.names=TRUE)
}


── Column specification ────────────────────────────────────────────────────────
cols(
  gene_names = col_character(),
  `0` = col_double(),
  `1` = col_double(),
  wilcoxon_logfc = col_double(),
  wilcoxon_padj = col_double(),
  wilcoxon_score = col_double(),
  `t-test_logfc` = col_double(),
  `t-test_padj` = col_double(),
  `t-test_score` = col_double(),
  `t-test_overestim_var_logfc` = col_double(),
  `t-test_overestim_var_padj` = col_double(),
  `t-test_overestim_var_score` = col_double()
)


Warning message:
“Unknown columns: `gene_name`”


[1] 4233



── Column specification ────────────────────────────────────────────────────────
cols(
  gs_name = col_character(),
  gene_symbol = col_character()
)


preparing geneSet collections...

calculating observed enrichment scores...

calculating permutation scores...

calculating p values...

leading edge analysis...

done...


── Column specification ────────────────────────────────────────────────────────
cols(
  gene_names = col_character(),
  `0` = col_double(),
  `10` = col_double(),
  wilcoxon_logfc = col_double(),
  wilcoxon_padj = col_double(),
  wilcoxon_score = col_double(),
  `t-test_logfc` = col_double(),
  `t-test_padj` = col_double(),
  `t-test_score` = col_double(),
  `t-test_overestim_var_logfc` = col_double(),
  `t-test_overestim_var_padj` = col_double(),
  `t-test_overestim_var_score` = col_double()
)


Warning message:
“Unknown columns: `gene_name`”


[1] 5227



── Column specification ────────────────────────────────────────────────────────
cols(
  gs_name = col_character(),
  gene_symbol = col_character()
)


preparing geneSet collections...

calculating observed enrichment scores...

calculating permutation scores...

calculating p values...

leading edge analysis...

done...


── Column specification ────────────────────────────────────────────────────────
cols(
  gene_names = col_character(),
  `0` = col_double(),
  `2` = col_double(),
  wilcoxon_logfc = col_double(),
  wilcoxon_padj = col_double(),
  wilcoxon_score = col_double(),
  `t-test_logfc` = col_double(),
  `t-test_padj` = col_double(),
  `t-test_score` = col_double(),
  `t-test_overestim_var_logfc` = col_double(),
  `t-test_overestim_var_padj` = col_double(),
  `t-test_overestim_var_score` = col_double()
)


Warning message:
“Unknown columns: `gene_name`”


...


── Column specification ────────────────────────────────────────────────────────
cols(
  gs_name = col_character(),
  gene_symbol = col_character()
)


preparing geneSet collections...

calculating observed enrichment scores...

calculating permutation scores...

calculating p values...

leading edge analysis...

done...


── Column specification ────────────────────────────────────────────────────────
cols(
  gene_names = col_character(),
  wilcoxon_logfc = col_double(),
  wilcoxon_padj = col_double(),
  wilcoxon_score = col_double(),
  `t-test_logfc` = col_double(),
  `t-test_padj` = col_double(),
  `t-test_score` = col_double(),
  `t-test_overestim_var_logfc` = col_double(),
  `t-test_overestim_var_padj` = col_double(),
  `t-test_overestim_var_score` = col_double(),
  logreg_score = col_double()
)


Warning message:
“Unknown columns: `gene_name`”


[1] 9132



── Column specification ────────────────────────────────────────────────────────
cols(
  gs_name = col_character(),
  gene_symbol = col_character()
)


preparing geneSet collections...

calculating observed enrichment scores...

calculating permutation scores...

calculating p values...

leading edge analysis...

done...


── Column specification ────────────────────────────────────────────────────────
cols(
  gene_names = col_character(),
  wilcoxon_logfc = col_double(),
  wilcoxon_padj = col_double(),
  wilcoxon_score = col_double(),
  `t-test_logfc` = col_double(),
  `t-test_padj` = col_double(),
  `t-test_score` = col_double(),
  `t-test_overestim_var_logfc` = col_double(),
  `t-test_overestim_var_padj` = col_double(),
  `t-test_overestim_var_score` = col_double(),
  logreg_score = col_double()
)


Warning message:
“Unknown columns: `gene_name`”


[1] 4319



── Column specification ────────────────────────────────────────────────────────
cols(
  gs_name = col_character(),
  gene_symbol = col_character()
)


preparing geneSet collections...

calculating observed enrichment scores...

calculating permutation scores...

calculating p values...

leading edge analysis...

done...

